# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

#### LIBRARY LOADER

In [1]:
import torch
import torchvision
import pandas as pd
import numpy as np

import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision import datasets, transforms
import torchvision.models as models

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### FUNCTIONS

In [2]:
# TODO: Define your transforms for the training, validation, and testing sets
def load_train_data(train_dir, bs=64, Prob_flip=0, Rot_deg=0):
    train_transforms = transforms.Compose([transforms.CenterCrop(255),
                                     transforms.Resize(224),
                                     transforms.RandomHorizontalFlip(Prob_flip),
                                     transforms.RandomRotation(Rot_deg),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    
    train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=bs, shuffle=True)
    
    return trainloader

In [3]:
def load_vt_data(train_dir,val_dir,bs=64):
    vt_transforms = transforms.Compose([transforms.CenterCrop(255),
                                    transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    
    test_data = datasets.ImageFolder(test_dir , transform=vt_transforms)
    val_data = datasets.ImageFolder(val_dir , transform=vt_transforms)
    
    testloader = torch.utils.data.DataLoader(test_data, batch_size=bs)
    valloader = torch.utils.data.DataLoader(val_data, batch_size=bs)
    
    return testloader, valloader

In [4]:
def validation(model, vtloader, crit, dev):
    vt_loss = 0
    accuracy = 0
    for jj, (images, labels) in enumerate(vtloader):
        
        images = images.to(device)
        labels = labels.to(device)

        output = model.forward(images)
        vt_loss += crit(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return vt_loss, accuracy

In [5]:
def train_mod(model, trn_ldr, v_ldr, dev, opt, crit, epochs=1, print_every=10):
    steps = 0
    for e in range(epochs):
        model.to(dev)
        model.train()
        running_loss = 0
    
        for ii, (images, labels) in enumerate(trn_ldr):
            images = images.to(dev)
            labels = labels.to(dev)
        
            steps += 1
                       
            opt.zero_grad()
        
            output = model.forward(images)
            loss = crit(output, labels)
            loss.backward()
            opt.step()
            running_loss += loss.item()
              
        
            if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                model.eval()

                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    val_loss, accuracy = validation(model, v_ldr, crit, dev)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Cross_val Loss: {:.3f}.. ".format(val_loss/len(v_ldr)),
                      "Val Accuracy: {:.3f}".format(accuracy/len(v_ldr)))

                running_loss = 0

                # Make sure training is back on
                model.train()
    
    return 

In [6]:
def add_Resnetclass(model,output,h_layers,p_drop=0,smax_dim=1):
        
    from collections import OrderedDict
    
    hidden_layer = []
    hidden_layer.extend(h_layers)
    hidden_layer.insert(0,model.fc.in_features)
    hidden_layer.append(output)
    print(hidden_layer)
    
    io_pairs = zip(hidden_layer[:-1],hidden_layer[1:])
    print(io_pairs)
    l_dict = OrderedDict()
    
    l = 0
    for a,b in io_pairs:
        l_dict['fc'+str(l)] = nn.Linear(a,b)
        l_dict['relu'+str(l)] = nn.ReLU()
        if b!=output: l_dict['drop'+str(l)] = nn.Dropout(p_drop)
        l += 1
        
    l_dict['softmax'] = nn.LogSoftmax(dim=smax_dim)
    
    for param in model.parameters():
        param.requires_grad = False

    classifier = nn.Sequential(l_dict)

    model.fc = classifier
    
    for param in model.fc.parameters():
        param.requires_grad = True
    
    return model

In [7]:
def save_res18_mod(filename, model,n_classes,h_layers,p_drop):
    checkpoint = {'output_size': n_classes,
                  'hidden_layers': h_layers,
                  'p_drop' : p_drop,
                  'state_dict': model.state_dict()}
    torch.save(checkpoint, filename)
    print('Checkpoint saved: ', filename)
    
    return 

In [8]:
def load_res18_mod(filename):
    cp = torch.load(filename)  # loadcheck point
    
    Res18 = models.resnet18(pretrained=True)
    add_Resnetclass(Res18,cp['output_size'],cp['hidden_layers'],p_drop=cp['p_drop'])
    
    Res18.load_state_dict(cp['state_dict'])
        
    return Res18

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [9]:
bz = 64  # batch_size
data_dir = 'flowers'
train_dir = data_dir + '/train'
val_dir = data_dir + '/valid'
test_dir = data_dir + '/test'


In [10]:
trainloader = load_train_data(train_dir,bz,0,0)
testloader,valloader = load_vt_data(test_dir,val_dir,bz)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [11]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [12]:
pdCat = pd.DataFrame.from_dict(cat_to_name,orient='index')
pdCat.head(5)

,0
21,fire lily
3,canterbury bells
45,bolero deep blue
1,pink primrose
34,mexican aster


In [13]:
n_classes = pdCat.shape[0]


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [60]:
# TODO: Build and train your network

#### Model 1

In [23]:
Res18_1 = models.resnet18(pretrained=True)

#model1 
output = 102
h_layers = [512, 256, 128]
p_drop = 0
lr = 0.001



In [24]:
h_layers

[512, 256, 128]

In [25]:
add_Resnetclass(Res18_1,output,h_layers,p_drop=p_drop)
Res18_1.fc


[512, 512, 256, 128, 102]


Sequential(
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0)
  (fc3): Linear(in_features=128, out_features=102, bias=True)
  (relu3): ReLU()
  (softmax): LogSoftmax()
)

In [26]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(Res18_1.fc.parameters(), lr=lr)

In [27]:
train_mod(Res18_1,trainloader,valloader,device,optimizer,criterion,epochs=5,print_every=20)

Epoch: 1/5..  Training Loss: 4.542..  Cross_val Loss: 4.456..  Val Accuracy: 0.101
Epoch: 1/5..  Training Loss: 4.313..  Cross_val Loss: 4.227..  Val Accuracy: 0.108
Epoch: 1/5..  Training Loss: 4.038..  Cross_val Loss: 3.954..  Val Accuracy: 0.194
Epoch: 1/5..  Training Loss: 3.881..  Cross_val Loss: 3.798..  Val Accuracy: 0.240
Epoch: 1/5..  Training Loss: 3.715..  Cross_val Loss: 3.732..  Val Accuracy: 0.239
Epoch: 2/5..  Training Loss: 2.870..  Cross_val Loss: 3.571..  Val Accuracy: 0.287
Epoch: 2/5..  Training Loss: 3.520..  Cross_val Loss: 3.538..  Val Accuracy: 0.293
Epoch: 2/5..  Training Loss: 3.372..  Cross_val Loss: 3.453..  Val Accuracy: 0.310
Epoch: 2/5..  Training Loss: 3.376..  Cross_val Loss: 3.442..  Val Accuracy: 0.310
Epoch: 2/5..  Training Loss: 3.277..  Cross_val Loss: 3.386..  Val Accuracy: 0.314
Epoch: 3/5..  Training Loss: 2.261..  Cross_val Loss: 3.403..  Val Accuracy: 0.312
Epoch: 3/5..  Training Loss: 3.126..  Cross_val Loss: 3.383..  Val Accuracy: 0.319
Epoc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [29]:
trainloader = load_train_data(train_dir,bz,0.50,90)

In [30]:
train_mod(Res18_1,trainloader,valloader,device,optimizer,criterion,epochs=1,print_every=20)

Epoch: 1/1..  Training Loss: 3.371..  Cross_val Loss: 3.538..  Val Accuracy: 0.310
Epoch: 1/1..  Training Loss: 3.331..  Cross_val Loss: 3.384..  Val Accuracy: 0.323
Epoch: 1/1..  Training Loss: 3.334..  Cross_val Loss: 3.449..  Val Accuracy: 0.311
Epoch: 1/1..  Training Loss: 3.343..  Cross_val Loss: 3.537..  Val Accuracy: 0.309
Epoch: 1/1..  Training Loss: 3.165..  Cross_val Loss: 3.504..  Val Accuracy: 0.319


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [33]:
Res18_1.fc.drop0.p = 0.2
Res18_1.fc.drop1.p = 0.2
Res18_1.fc.drop2.p = 0.2
Res18_1.fc

Sequential(
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0.2)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.2)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.2)
  (fc3): Linear(in_features=128, out_features=102, bias=True)
  (relu3): ReLU()
  (softmax): LogSoftmax()
)

In [34]:
train_mod(Res18_1,trainloader,valloader,device,optimizer,criterion,epochs=3,print_every=20)

Epoch: 1/3..  Training Loss: 3.388..  Cross_val Loss: 3.425..  Val Accuracy: 0.317
Epoch: 1/3..  Training Loss: 3.388..  Cross_val Loss: 3.361..  Val Accuracy: 0.333
Epoch: 1/3..  Training Loss: 3.333..  Cross_val Loss: 3.344..  Val Accuracy: 0.327
Epoch: 1/3..  Training Loss: 3.441..  Cross_val Loss: 3.323..  Val Accuracy: 0.337
Epoch: 1/3..  Training Loss: 3.322..  Cross_val Loss: 3.348..  Val Accuracy: 0.327
Epoch: 2/3..  Training Loss: 2.780..  Cross_val Loss: 3.327..  Val Accuracy: 0.332
Epoch: 2/3..  Training Loss: 3.371..  Cross_val Loss: 3.388..  Val Accuracy: 0.325
Epoch: 2/3..  Training Loss: 3.292..  Cross_val Loss: 3.303..  Val Accuracy: 0.339
Epoch: 2/3..  Training Loss: 3.352..  Cross_val Loss: 3.281..  Val Accuracy: 0.337
Epoch: 2/3..  Training Loss: 3.314..  Cross_val Loss: 3.336..  Val Accuracy: 0.337
Epoch: 3/3..  Training Loss: 2.262..  Cross_val Loss: 3.281..  Val Accuracy: 0.340
Epoch: 3/3..  Training Loss: 3.397..  Cross_val Loss: 3.286..  Val Accuracy: 0.341
Epoc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [35]:
save_res18_mod('model1.pth',Res18_1,n_classes,h_layers,0.2)

Checkpoint saved:  model1.pth


In [14]:
Res18_2 = models.resnet18(pretrained=True)

#model1 
output = 102
h_layers = [256]
p_drop = 0
lr = 0.001

add_Resnetclass(Res18_2,output,h_layers,p_drop=p_drop)
Res18_2.fc

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:01<00:00, 38711610.80it/s]


[512, 256, 102]


Sequential(
  (fc0): Linear(in_features=512, out_features=256, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0)
  (fc1): Linear(in_features=256, out_features=102, bias=True)
  (relu1): ReLU()
  (softmax): LogSoftmax()
)

In [15]:
criterion2 = nn.NLLLoss()
optimizer2 = optim.Adam(Res18_2.fc.parameters(), lr=lr)

In [16]:
trainloader = load_train_data(train_dir,bz,0,0)

In [17]:
train_mod(Res18_2,trainloader,valloader,device,optimizer2,criterion2,epochs=4,print_every=20)

Epoch: 1/4..  Training Loss: 4.477..  Cross_val Loss: 4.285..  Val Accuracy: 0.174
Epoch: 1/4..  Training Loss: 4.115..  Cross_val Loss: 3.946..  Val Accuracy: 0.213
Epoch: 1/4..  Training Loss: 3.926..  Cross_val Loss: 3.722..  Val Accuracy: 0.271
Epoch: 1/4..  Training Loss: 3.678..  Cross_val Loss: 3.572..  Val Accuracy: 0.302
Epoch: 1/4..  Training Loss: 3.502..  Cross_val Loss: 3.528..  Val Accuracy: 0.297
Epoch: 2/4..  Training Loss: 2.944..  Cross_val Loss: 3.465..  Val Accuracy: 0.307
Epoch: 2/4..  Training Loss: 3.325..  Cross_val Loss: 3.423..  Val Accuracy: 0.306
Epoch: 2/4..  Training Loss: 3.425..  Cross_val Loss: 3.378..  Val Accuracy: 0.315
Epoch: 2/4..  Training Loss: 3.265..  Cross_val Loss: 3.356..  Val Accuracy: 0.315
Epoch: 2/4..  Training Loss: 3.361..  Cross_val Loss: 3.330..  Val Accuracy: 0.317
Epoch: 3/4..  Training Loss: 2.335..  Cross_val Loss: 3.350..  Val Accuracy: 0.325
Epoch: 3/4..  Training Loss: 3.193..  Cross_val Loss: 3.333..  Val Accuracy: 0.316
Epoc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [18]:
trainloader = load_train_data(train_dir,bz,0.25,12.5)

In [19]:
train_mod(Res18_2,trainloader,valloader,device,optimizer2,criterion2,epochs=4,print_every=20)

Epoch: 1/4..  Training Loss: 3.290..  Cross_val Loss: 3.389..  Val Accuracy: 0.316
Epoch: 1/4..  Training Loss: 3.242..  Cross_val Loss: 3.380..  Val Accuracy: 0.307
Epoch: 1/4..  Training Loss: 3.294..  Cross_val Loss: 3.344..  Val Accuracy: 0.316
Epoch: 1/4..  Training Loss: 3.279..  Cross_val Loss: 3.350..  Val Accuracy: 0.321
Epoch: 1/4..  Training Loss: 3.398..  Cross_val Loss: 3.341..  Val Accuracy: 0.320
Epoch: 2/4..  Training Loss: 2.787..  Cross_val Loss: 3.338..  Val Accuracy: 0.317
Epoch: 2/4..  Training Loss: 3.191..  Cross_val Loss: 3.378..  Val Accuracy: 0.312
Epoch: 2/4..  Training Loss: 3.172..  Cross_val Loss: 3.341..  Val Accuracy: 0.320
Epoch: 2/4..  Training Loss: 3.297..  Cross_val Loss: 3.312..  Val Accuracy: 0.326
Epoch: 2/4..  Training Loss: 3.213..  Cross_val Loss: 3.345..  Val Accuracy: 0.329
Epoch: 3/4..  Training Loss: 2.178..  Cross_val Loss: 3.322..  Val Accuracy: 0.327
Epoch: 3/4..  Training Loss: 3.249..  Cross_val Loss: 3.314..  Val Accuracy: 0.325
Epoc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [21]:
p_drop = 0.2
Res18_2.fc.drop0.p = p_drop
Res18_2.fc

Sequential(
  (fc0): Linear(in_features=512, out_features=256, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0.2)
  (fc1): Linear(in_features=256, out_features=102, bias=True)
  (relu1): ReLU()
  (softmax): LogSoftmax()
)

In [22]:
trainloader = load_train_data(train_dir,bz,0.25,12.5)

In [23]:
train_mod(Res18_2,trainloader,valloader,device,optimizer2,criterion2,epochs=4,print_every=20)

Epoch: 1/4..  Training Loss: 3.224..  Cross_val Loss: 3.279..  Val Accuracy: 0.331
Epoch: 1/4..  Training Loss: 3.157..  Cross_val Loss: 3.299..  Val Accuracy: 0.320
Epoch: 1/4..  Training Loss: 3.236..  Cross_val Loss: 3.301..  Val Accuracy: 0.332
Epoch: 1/4..  Training Loss: 3.249..  Cross_val Loss: 3.303..  Val Accuracy: 0.328
Epoch: 1/4..  Training Loss: 3.249..  Cross_val Loss: 3.287..  Val Accuracy: 0.333
Epoch: 2/4..  Training Loss: 2.696..  Cross_val Loss: 3.280..  Val Accuracy: 0.332
Epoch: 2/4..  Training Loss: 3.147..  Cross_val Loss: 3.266..  Val Accuracy: 0.332
Epoch: 2/4..  Training Loss: 3.173..  Cross_val Loss: 3.291..  Val Accuracy: 0.319
Epoch: 2/4..  Training Loss: 3.283..  Cross_val Loss: 3.281..  Val Accuracy: 0.323
Epoch: 2/4..  Training Loss: 3.217..  Cross_val Loss: 3.284..  Val Accuracy: 0.327
Epoch: 3/4..  Training Loss: 2.215..  Cross_val Loss: 3.281..  Val Accuracy: 0.329
Epoch: 3/4..  Training Loss: 3.206..  Cross_val Loss: 3.274..  Val Accuracy: 0.330
Epoc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [17]:
Res18_1A = load_res18_mod('model1.pth')

[512, 512, 256, 128, 102]


In [20]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(Res18_1A.fc.parameters(), lr=0.001)

Sequential(
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0.2)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.2)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.2)
  (fc3): Linear(in_features=128, out_features=102, bias=True)
  (relu3): ReLU()
  (softmax): LogSoftmax()
)

In [22]:
train_mod(Res18_1A,trainloader,valloader,device,optimizer,criterion,epochs=4,print_every=20)

Epoch: 1/4..  Training Loss: 3.118..  Cross_val Loss: 3.188..  Val Accuracy: 0.349
Epoch: 1/4..  Training Loss: 3.051..  Cross_val Loss: 3.196..  Val Accuracy: 0.345
Epoch: 1/4..  Training Loss: 3.024..  Cross_val Loss: 3.181..  Val Accuracy: 0.349
Epoch: 1/4..  Training Loss: 2.997..  Cross_val Loss: 3.194..  Val Accuracy: 0.350
Epoch: 1/4..  Training Loss: 3.138..  Cross_val Loss: 3.164..  Val Accuracy: 0.356
Epoch: 2/4..  Training Loss: 2.573..  Cross_val Loss: 3.173..  Val Accuracy: 0.351
Epoch: 2/4..  Training Loss: 3.046..  Cross_val Loss: 3.158..  Val Accuracy: 0.351
Epoch: 2/4..  Training Loss: 2.951..  Cross_val Loss: 3.157..  Val Accuracy: 0.353
Epoch: 2/4..  Training Loss: 3.022..  Cross_val Loss: 3.178..  Val Accuracy: 0.343
Epoch: 2/4..  Training Loss: 3.084..  Cross_val Loss: 3.173..  Val Accuracy: 0.344
Epoch: 3/4..  Training Loss: 2.108..  Cross_val Loss: 3.165..  Val Accuracy: 0.355
Epoch: 3/4..  Training Loss: 2.987..  Cross_val Loss: 3.184..  Val Accuracy: 0.350
Epoc

In [24]:
## MOdel #3
Res18_3 = models.resnet18(pretrained=True)

#model1 
output = 102
h_layers = [512, 512, 512, 256]
p_drop = 0
lr = 0.005

add_Resnetclass(Res18_3,output,h_layers,p_drop=p_drop)
Res18_3.fc


[512, 512, 512, 512, 256, 102]


Sequential(
  (fc0): Linear(in_features=512, out_features=512, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (relu3): ReLU()
  (drop3): Dropout(p=0)
  (fc4): Linear(in_features=256, out_features=102, bias=True)
  (relu4): ReLU()
  (softmax): LogSoftmax()
)

In [26]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(Res18_3.fc.parameters(), lr=0.005)

In [27]:
train_mod(Res18_3,trainloader,valloader,device,optimizer,criterion,epochs=4,print_every=20)

Epoch: 1/4..  Training Loss: 4.599..  Cross_val Loss: 4.500..  Val Accuracy: 0.029
Epoch: 1/4..  Training Loss: 4.441..  Cross_val Loss: 4.432..  Val Accuracy: 0.049
Epoch: 1/4..  Training Loss: 4.395..  Cross_val Loss: 4.345..  Val Accuracy: 0.060
Epoch: 1/4..  Training Loss: 4.265..  Cross_val Loss: 4.235..  Val Accuracy: 0.077
Epoch: 1/4..  Training Loss: 4.246..  Cross_val Loss: 4.194..  Val Accuracy: 0.088
Epoch: 2/4..  Training Loss: 3.561..  Cross_val Loss: 4.094..  Val Accuracy: 0.123
Epoch: 2/4..  Training Loss: 4.056..  Cross_val Loss: 3.990..  Val Accuracy: 0.129
Epoch: 2/4..  Training Loss: 3.958..  Cross_val Loss: 3.946..  Val Accuracy: 0.161
Epoch: 2/4..  Training Loss: 3.937..  Cross_val Loss: 3.909..  Val Accuracy: 0.162
Epoch: 2/4..  Training Loss: 4.017..  Cross_val Loss: 3.965..  Val Accuracy: 0.135
Epoch: 3/4..  Training Loss: 2.724..  Cross_val Loss: 3.892..  Val Accuracy: 0.172
Epoch: 3/4..  Training Loss: 3.831..  Cross_val Loss: 3.877..  Val Accuracy: 0.162
Epoc

In [29]:
Res18_3.fc.drop0.p = 0.5
Res18_3.fc.drop1.p = 0.5
Res18_3.fc.drop2.p = 0.5
Res18_3.fc.drop3.p = 0.5

In [32]:
train_mod(Res18_3,trainloader,valloader,device,optimizer,criterion,epochs=4,print_every=40)

Epoch: 1/4..  Training Loss: 4.481..  Cross_val Loss: 4.448..  Val Accuracy: 0.064
Epoch: 1/4..  Training Loss: 4.524..  Cross_val Loss: 4.451..  Val Accuracy: 0.059
Epoch: 2/4..  Training Loss: 1.935..  Cross_val Loss: 4.528..  Val Accuracy: 0.052
Epoch: 2/4..  Training Loss: 4.555..  Cross_val Loss: 4.507..  Val Accuracy: 0.049
Epoch: 2/4..  Training Loss: 4.538..  Cross_val Loss: 4.492..  Val Accuracy: 0.050
Epoch: 3/4..  Training Loss: 3.862..  Cross_val Loss: 4.415..  Val Accuracy: 0.067
Epoch: 3/4..  Training Loss: 4.553..  Cross_val Loss: 4.488..  Val Accuracy: 0.056
Epoch: 4/4..  Training Loss: 1.241..  Cross_val Loss: 4.478..  Val Accuracy: 0.054
Epoch: 4/4..  Training Loss: 4.541..  Cross_val Loss: 4.491..  Val Accuracy: 0.043
Epoch: 4/4..  Training Loss: 4.529..  Cross_val Loss: 4.574..  Val Accuracy: 0.061


In [34]:
## MOdel #4
Res18_4 = models.resnet18(pretrained=True)

#model1 
output = 102
h_layers = [256, 128]
p_drop = 0
lr = 0.005

add_Resnetclass(Res18_4,output,h_layers,p_drop=p_drop)
Res18_4.fc

[512, 256, 128, 102]


Sequential(
  (fc0): Linear(in_features=512, out_features=256, bias=True)
  (relu0): ReLU()
  (drop0): Dropout(p=0)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0)
  (fc2): Linear(in_features=128, out_features=102, bias=True)
  (relu2): ReLU()
  (softmax): LogSoftmax()
)

In [35]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(Res18_4.fc.parameters(), lr=0.005)

In [36]:
train_mod(Res18_4,trainloader,valloader,device,optimizer,criterion,epochs=4,print_every=40)

Epoch: 1/4..  Training Loss: 4.246..  Cross_val Loss: 3.892..  Val Accuracy: 0.194
Epoch: 1/4..  Training Loss: 3.773..  Cross_val Loss: 3.694..  Val Accuracy: 0.249
Epoch: 2/4..  Training Loss: 1.509..  Cross_val Loss: 3.552..  Val Accuracy: 0.276
Epoch: 2/4..  Training Loss: 3.479..  Cross_val Loss: 3.515..  Val Accuracy: 0.284
Epoch: 2/4..  Training Loss: 3.454..  Cross_val Loss: 3.405..  Val Accuracy: 0.298
Epoch: 3/4..  Training Loss: 2.842..  Cross_val Loss: 3.394..  Val Accuracy: 0.303
Epoch: 3/4..  Training Loss: 3.403..  Cross_val Loss: 3.516..  Val Accuracy: 0.292
Epoch: 4/4..  Training Loss: 0.869..  Cross_val Loss: 3.396..  Val Accuracy: 0.305
Epoch: 4/4..  Training Loss: 3.280..  Cross_val Loss: 3.390..  Val Accuracy: 0.300
Epoch: 4/4..  Training Loss: 3.370..  Cross_val Loss: 3.359..  Val Accuracy: 0.305


In [ ]:
Res18_4.fc.

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

epochs = 5
steps = 0
print_every = 10
Res18_0 = Res18_0.to(device)

for e in range(epochs):
    Res18_0.train()
    running_loss = 0
    
    for ii, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        
        steps += 1
                       
        optimizer.zero_grad()
        
        output = Res18_0.forward(images)
        loss = criterion(output, labels)
        #loss.requires_grad = True
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # Make sure network is in eval mode for inference
            Res18_0.eval()
            
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                test_loss, accuracy = validation(Res18_0, testloader, criterion, device)
           
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
            
            running_loss = 0
            
            # Make sure training is back on
            Res18_0.train()